<h1> Homework 10: Numerical PDEs <h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Problem 3

Let's tackle the celebrated N-body problem where one studies the evolution of **N** particles that are interacting with a force that depends only on the nature of the particles and the distance between them. Applications includes star evolution in galaxies, solar systems, molecular dynamics, etc...

(a) Write a function **step_update(pos, vel, acc, force, types, masses, dt, box_size)** that evolves an **N** particle sytem forward in time by a small time step **dt**.

The potential we will use is Newtonian. To advance a time step, use the following simple algorithm

$$
r_i = r_i + v_i\,dt + \frac{1}{2}\,a_i\,dt^2
$$

where $i$ ranges between **1** and **d** for each particle. And

$$
v_i = v_i + \frac{1}{2} \left(a_i+a_i^{new}\right)\,dt
$$

where $a_i^{new}$ is the new acceleration of the particle computed from current force divided mass.

Once these are computed, update the acceleration for each particle with force divided by mass. The function should also check that the particles stay in the box and reflect their velocities and accelerations when appropriate.

In [31]:
def step_update(pos, vel, acc, force, types, masses, dt, box_size):
    # Inputs:
    # 
    # pos, vel, acc, and force are N by d matrices, where d is 
    # the dimension of space (typically 3), representing respectively the positions, 
    # velocities, accelerations, and forces on each particle.
    # types is an N dimensional array identifying the type of each particle 
    # by an integer; this integer is an index in the masses array to identify 
    # the masses of the types of particles. For example, for a single type of 
    # molecule, the masses array would have one element, and the types array 
    # would be N ones.
    # masses is an array of masses; the size of the array corresponds to the 
    # number of types of particles on may have, as described in the previous 
    # paragraph.
    # dt is the time step to advance.
    # box_size is related to the size of the d dimensional cube in which the 
    # particles are confined; for example, the x coordinate would range between 
    # -box_size and +box_size. The particles reflect elastically on the walls of 
    # the box.
    # 
    # Outputs:
    # 
    # The function does not return anything. Instead, it updates the pos, vel, 
    # and acc matrices.

    d = len(pos[0])
    N = len(pos)
    
    pos += vel*dt + 0.5*acc*dt*dt
    acc_new = force/np.array([[masses[t-1]] for t in types])
    vel += 0.5*(acc + acc_new)*dt
    acc = acc_new
    # check all particles are in the box
    for i in range(N):
        for j in range(d):
            if pos[i,j] < 0:
                pos[i,j] = -pos[i,j]
                vel[i,j] = -vel[i,j]
                acc[i,j] = -acc[i,j]
            if pos[i,j] >= box_size:
                pos[i,j] = 2*box_size - pos[i,j]
                vel[i,j] = -vel[i,j]
                acc[i,j] = -acc[i,j]

(b) Write a function compute_force(pos, vel, acc, interactions) which computes the force on each particle and returns an N by d array of these forces.

In [34]:
def compute_force(pos, vel, acc, interactions):

    # Inputs:
    # 
    # pos, vel, and acc are as described above.
    # interactions is a square matrix which gives the interactions between any 
    # two types of particles. For example, if we have two types of particles with 
    # masses m1 and m2, and we use gravitational or coulombic forces between the 
    # particles, the force between two particles labeled i and j of type 1 and 
    # type 2 respectively would be
    # 
    # -interactions[1,2]/rij^2
    # 
    # where rij is the Euclidean distance between particles i and j. 
    # We include vel and acc to allow for modifications that include velocity 
    # and acceleration-dependent forces.
    #
    # Outputs:
    #
    # A N by d array of forces

    d = len(pos[0])
    N = len(pos)
    force = np.zeros((N,d))
    
    for i in range(N):
        for j in range(d):
            rsquared = np.linalg.norm(pos[i]-pos[j])**2
            force_ij = -interactions[i,j]/rsquared
            force[i] += force_ij*(pos[i]-pos[j])/np.linalg.norm(pos[i]-pos[j])
            force[j] -= force_ij*(pos[i]-pos[j])/np.linalg.norm(pos[i]-pos[j])
    return force

(c) Write a simulation function simulate(dt, tspan, pos0, vel0, types, masses, interactions) that runs the simulation with time step dt for a time interval tspan=(t0,tmax), initial position and velocity arrays given by pos0 and vel0, and the types, masses, and interactions arrays as described above

In [35]:
def simulate(dt, tspan, pos0, vel0, types, masses, interactions):

    # Inputs:
    # 
    # dt: Time step
    # tspan: A tuple (t0,tmax).
    # pos0 and vel0: Initial position and velocity arrays.
    # types, masses, and interactions: As described earlier.
    # 
    # Outputs:
    # 
    # The function should return an array of time snapshots of the positions 
    # for all particles.
    
    nsteps = int((tspan[1]-tspan[0])/dt)
    d = len(pos0[0])
    N = len(pos0)
    pos = pos0.copy()
    vel = vel0.copy()
    acc = np.zeros((N,d))
    
    for step in range(nsteps):
        force = compute_force(pos, vel, acc, interactions)
        step_update(pos, vel, acc, force, types, masses, dt, 1)
    return pos

In [40]:
pos0 = np.matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
vel0 = np.matrix([[-0.1, 0.0, 1.0], [1.0, -0.1, 0.0], [0.0, 1.0, -0.1]])
types = [1, 2, 3]
masses = [1.0, 0.5, 2.0]
interactions = np.matrix([[1.0*masses[i]*masses[j] for i in range(len(masses))] for j in range(len(masses))])
force = pos0
dt = 0.01
box_size = 10.0
acc0 = force/np.array([[masses[t-1]] for t in types])

In [41]:
pos = simulate(dt, (0, 10), pos0, vel0, types, masses, interactions)

/var/folders/2f/cvmbnp4s1r362w323xc1fx800000gn/T/ipykernel_24478/4290503901.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  force_ij = -interactions[i,j]/rsquared
/var/folders/2f/cvmbnp4s1r362w323xc1fx800000gn/T/ipykernel_24478/4290503901.py:30: RuntimeWarning: invalid value encountered in multiply
  force[i] += force_ij*(pos[i]-pos[j])/np.linalg.norm(pos[i]-pos[j])


ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (1,3)